In [104]:
import pandas as pd
import folium
import numpy as np
import os
import googlemaps
import requests
from textwrap import shorten
from bs4 import BeautifulSoup
import re

In [105]:
df = pd.read_csv('FortLauderdaleCoastalBrowardCondoMap - BrowardCondoPipelineMap.csv',dtype='str')

## Data clean

In [106]:
df = df.dropna(subset='Address')

In [107]:
df = df.drop(columns=['Developer Checked','Emailed PR'])

In [108]:
df['Full_Address'] = df['Address'] + ' ' + df['City'] + ' , FL'

## Geocode

In [109]:
%store -r google_maps_API_Key
gmaps_key = googlemaps.Client(key=google_maps_API_Key)

In [110]:
# Define the geocode function
def geocode(add):
    g = gmaps_key.geocode(add)
    if g:
        lat = g[0]["geometry"]["location"]["lat"]
        lng = g[0]["geometry"]["location"]["lng"]
        return (lat, lng)
    else:
        return None

# Apply geocoding to the 'geo_address' column and store the results in 'geocoded' column
df['geocoded'] = df['Full_Address'].apply(geocode)

## Geocoded

In [120]:
%store -r google_maps_API_Key
gmaps_key = googlemaps.Client(key=google_maps_API_Key)

In [116]:
df['Full_Address'] = df['Address'] + ' ' + df['City'] + ', FL'

In [126]:
# Define the geocode function
def geocode(add):
    g = gmaps_key.geocode(add)
    if g:
        lat = g[0]["geometry"]["location"]["lat"]
        lng = g[0]["geometry"]["location"]["lng"]
        return (lat, lng)
    else:
        return None

# Apply geocoding to the 'geo_address' column and store the results in 'geocoded' column
df['geocoded'] = df['Full_Address'].apply(geocode)

In [127]:
df['geocoded'] = df['geocoded'].astype(str)
df[['lat', 'lon']] = df['geocoded'].apply(lambda x: (None, None) if x == 'None' else x.strip('()').split(', ', 1)).apply(pd.Series)
df['lat'] = df['lat'].astype(float)
df['lon'] = df['lon'].astype(float)

## Link Formatter

In [128]:
df['Story Link'] = 'Click here for story'

df["short_description"] = df["Story Link"].apply(lambda s: shorten(s, width=20, placeholder=""))
df['remaining_desc'] = df.apply(lambda row : row['Story Link'].replace(str(row['short_description']), ''), axis=1)
df['remaining_desc']

df["Story Link"] = '<a href="' + df["URL"] + '" target="_blank" rel="noopener noreferrer">' + df["short_description"] + "</a>" + df["remaining_desc"]


In [132]:
df.columns

Index(['Address', 'Project name', 'Developer', 'Description', 'City', 'URL',
       'Full_Address', 'geocoded', 'Story Link', 'short_description',
       'remaining_desc', 'lat', 'lon'],
      dtype='object')

In [141]:
import folium
from folium.plugins import MarkerCluster
import pandas as pd
import textwrap
import html

# Handle missing data and ensure all fields are strings
df["Address"] = df["Address"].fillna("N/A").astype(str)
df["Project name"] = df["Project name"].fillna("N/A").astype(str)
df["Developer"] = df["Developer"].fillna("N/A").astype(str)
df["Full_Address"] = df["Full_Address"].fillna("N/A").astype(str)
df["Description"] = df["Description"].fillna("N/A").astype(str)
df["URL"] = df["URL"].fillna("#").astype(str)

# Function to sanitize text to prevent HTML injection
def sanitize(text):
    return html.escape(text, quote=True)

# Apply sanitization
df["Address"] = df["Address"].apply(sanitize)
df["Project name"] = df["Project name"].apply(sanitize)
df["Developer"] = df["Developer"].apply(sanitize)
df["Full_Address"] = df["Full_Address"].apply(sanitize)
df["Description"] = df["Description"].apply(sanitize)
df["URL"] = df["URL"].apply(sanitize)

# Shorten the Description to create 'short_description'
df["short_description"] = df["Description"].apply(
    lambda s: textwrap.shorten(s, width=20, placeholder="...")
)

# Create 'remaining_desc' by removing 'short_description' from 'Description'
df['remaining_desc'] = df.apply(
    lambda row: row['Description'].replace(row['short_description'], '', 1), axis=1
)

# Create 'description_link' with the sanitized data
df["description_link"] = (
    f'<a href="{df["URL"]}" target="_blank" rel="noopener noreferrer">' +
    df["short_description"] +
    "</a>" +
    df["remaining_desc"]
)

# Define the popup HTML function
def popup_html(row_index, max_chars=30):
    """Generate HTML content for the popup with line breaks for long text."""
    popup_content = '<div class="popup-content">'

    # Specify the columns you want to include in the popup
    display_columns = ['Developer', 'Project name', 'Full_Address', 'Units', 'Stories', 'Est. Completion']

    for column in display_columns:
        if column in df.columns:
            value = df.at[row_index, column]
            value_str = str(value).strip()
            if pd.notnull(value) and value_str != "":

                # Insert <br> tags for long 'Site Address' entries
                if len(value_str) > max_chars and column == "Site Address":
                    value_str = '<br>'.join(textwrap.wrap(value_str, width=max_chars, break_long_words=False))

                popup_content += f'<strong>{column}:</strong> {value_str}<br>'

    # Add the description link
    description_link = df.at[row_index, 'description_link']
    popup_content += f'<strong>Description:</strong> {description_link}'

    popup_content += '</div>'
    return popup_content

# Initialize the map centered around the mean latitude and longitude
map_center = [df['lat'].mean(), df['lon'].mean()]
m = folium.Map(location=map_center, zoom_start=14)

# Add the custom Mapbox tile layer
folium.TileLayer(
    tiles = 'https://api.mapbox.com/styles/v1/mapbox/streets-v11/tiles/256/{z}/{x}/{y}@2x?access_token=pk.eyJ1IjoidHJkZGF0YSIsImEiOiJjamc2bTc2YmUxY2F3MnZxZGh2amR2MTY5In0.QlOWqB-yQNrNlXD0KQ9IvQ',
    attr='Mapbox',
    name='Streets',
    overlay=False,
    control=True,
    show=True,
    min_zoom=11,
    max_zoom=20
).add_to(m)

# Add custom CSS to style the popups
custom_css = """
<style>
    .popup-content {
        max-width: 300px; /* Adjusts the maximum width of the popup */
        font-size: 12px;   /* Adjusts the font size */
        word-wrap: break-word; /* Ensures long words wrap to the next line */
    }
    .leaflet-popup-content-wrapper {
        background-color: white;
        border: 1px solid gray;
        border-radius: 3px;
        padding: 5px;
    }
    a {
        color: blue; /* Customize link color if desired */
        text-decoration: underline;
    }
</style>
"""

# Add the custom CSS to the map
m.get_root().html.add_child(folium.Element(custom_css))

# Create and add the title to the map
title_html = '''
<h3 align="center" style="font-size:16px"><b>Residential Projects Planned in West Palm Beach</b></h3>
'''
m.get_root().html.add_child(folium.Element(title_html))

# Initialize a MarkerCluster to handle multiple markers efficiently
marker_cluster = MarkerCluster().add_to(m)

# Loop through the DataFrame to create markers with popups
for i in range(len(df)):
    # Generate popup HTML content
    popup_content = popup_html(i)
    
    # Create a Popup object with an IFrame to control dimensions and embed HTML
    popup = folium.Popup(
        folium.IFrame(html=popup_content, width=300, height=100),
        max_width=300
    )
    
    # Create a CircleMarker with the popup and add it to the MarkerCluster
    folium.CircleMarker(
        location=[df.iloc[i]['lat'], df.iloc[i]['lon']],
        popup=popup,
        color='red',
        fill=True,
        fill_color='red',
        radius=7,
        weight=1,          # Thickness of the circle border
        opacity=0.8,       # Opacity of the circle border
        fill_opacity=0.6   # Opacity of the fill
    ).add_to(marker_cluster)

# Add layer control to toggle different layers
folium.LayerControl().add_to(m)

# Display the map
m


In [43]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/Alexander_Map
